### Importamos librerias

In [1]:
import os
from stable_baselines3 import PPO, DQN
from snakeEnv import *

pygame 2.1.0 (SDL 2.0.16, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
logPath = os.path.join('Training','Logs')
ppo_path = os.path.join('Training','models')

In [14]:
env = snakeEnv()
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=logPath)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [15]:
env.render_mode = 'human'
model.learn(total_timesteps=5000)
env.close()
model.save("snake_ppo_model")


human
Logging to Training\Logs\PPO_4
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
hum

KeyboardInterrupt: 

In [11]:
model = PPO.load("snake_ppo_model")
obs = env.reset()
for i in range(2000):
    env.render_mode = 'none'
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
      obs = env.reset()

human
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none


1

In [ ]:
env.render_mode = 'human'
model.learn(total_timesteps=500)
env.close()

In [ ]:
#sin visualizar
env.render_mode = 'none'
model.learn(total_timesteps=1000)